In [1]:
import importlib
import os
from pathlib import Path

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [2]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation')

In [3]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_node import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

from bsm_multi_agents.agents.pricing_validator_node import (
    pricing_validator_tool_node,
)

graph = StateGraph(WorkflowState)

graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)
graph.add_node("pricing_validator_tool", pricing_validator_tool_node)

graph.add_edge(START, "pricing_calculator_tool")
graph.add_edge("pricing_calculator_tool", "pricing_validator_tool")
graph.add_edge("pricing_validator_tool", END)

app = graph.compile()

In [4]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

In [5]:
final_state

{'messages': [],
 'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'greeks_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'validate_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv',
 'errors': [],
 'remaining_steps': 0}